# LLM Optimization Modelling Experiment

In [1]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [595]:
problem = '''You are the person in charge of packing in a large company. Your job is to skillfully pack items of various weights in a box with predetermined capacity. The objective is to use as few boxes as possible. There is a set of items and a set of boxes with an upper bound on the amount of boxes. Each of the items has a known weight. Each item is packed into one box and items are packed into the chosen boxes. Please formulate a mathematical optimization model for this problem.   '''

## 2. Ask for parameters

In [623]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate only the variables for this mathematical optimization problem. 
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [624]:
#Show the resopnse in a formatted way
Markdown(response.text)

Let:

*  $I$ be the set of items.
* $J$ be the set of boxes. 
* $w_i$ be the weight of item $i \in I$.
* $C$ be the capacity of each box.
* $N$ be the upper bound on the number of boxes. 
* $x_{ij}$ be a binary variable equal to 1 if item $i \in I$ is assigned to box $j \in J$, and 0 otherwise. 
* $y_j$ be a binary variable equal to 1 if box $j \in J$ is used, and 0 otherwise. 


# 2. Ask for objective

In [625]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please formulate only the objective function for this mathematical optimization problem."
prompt2 += problem + response.text
response2 = generative_multimodal_model.generate_content([prompt2])

In [626]:
Markdown(response2.text)

Minimize $\sum_{j \in J} y_j$ 


# 3. Ask for constraints

In [627]:
#Second prompt gets the output of the previous step and generates the code
prompt3 = "Please formulate only the constraints for this mathematical optimization problem."
prompt3 += problem + response.text + response2.text
response3 = generative_multimodal_model.generate_content([prompt3])

In [628]:
Markdown(response3.text)

Subject to:

* $\sum_{j \in J} x_{ij} = 1, \forall i \in I$ 
* $\sum_{i \in I} w_i x_{ij}  \leq C y_j, \forall j \in J$
* $\sum_{j \in J} y_j \leq N$
* $x_{ij} \in \{0,1\}, \forall i \in I, \forall j \in J$
* $y_{j} \in \{0,1\}, \forall j \in J$ 


## 3. Generate the pyomo code

In [629]:
#Second prompt gets the output of the previous step and generates the code
prompt4 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt4 += response.text + response2.text + response3.text 
response4 = generative_multimodal_model.generate_content([prompt4])

In [630]:
## Showing the code in a formatted way
Markdown(response4.text)

```python
import pyomo.environ as pyo

# Sample data (replace with your actual data)
items = ['A', 'B', 'C', 'D', 'E']  
weights = {'A': 4, 'B': 2, 'C': 7, 'D': 3, 'E': 5} 
box_capacity = 10
max_boxes = 3

# Model
model = pyo.ConcreteModel()

# Sets
model.I = pyo.Set(initialize=items) # set of items
model.J = pyo.RangeSet(1, max_boxes) # set of boxes

# Parameters
model.w = pyo.Param(model.I, initialize=weights)  # weight of each item
model.C = box_capacity # capacity of each box
model.N = max_boxes # maximum number of boxes

# Variables
model.x = pyo.Var(model.I, model.J, domain=pyo.Binary) # 1 if item i is in box j, 0 otherwise
model.y = pyo.Var(model.J, domain=pyo.Binary) # 1 if box j is used, 0 otherwise

# Objective function: Minimize the number of boxes used
model.obj = pyo.Objective(expr=sum(model.y[j] for j in model.J), sense=pyo.minimize)

# Constraints
# Each item must be assigned to exactly one box
model.constraint1 = pyo.ConstraintList()
for i in model.I:
    model.constraint1.add(sum(model.x[i, j] for j in model.J) == 1)

# Weight capacity constraint for each box
model.constraint2 = pyo.ConstraintList()
for j in model.J:
    model.constraint2.add(sum(model.w[i] * model.x[i, j] for i in model.I) <= model.C * model.y[j])

# Limit the total number of boxes used
model.constraint3 = pyo.Constraint(expr=sum(model.y[j] for j in model.J) <= model.N)

# Solve the model
solver = pyo.SolverFactory('glpk') 
results = solver.solve(model)

# Print the results
print("Status:", results.solver.status)
print("Termination condition:",results.solver.termination_condition)
print("Optimal number of boxes:", pyo.value(model.obj))

# Print box assignments
for j in model.J:
    if pyo.value(model.y[j]) > 0:
        print(f"Box {j}:", [i for i in model.I if pyo.value(model.x[i, j]) > 0.5])
```

**Explanation:**

1.  **Import Pyomo:** `import pyomo.environ as pyo`
2.  **Sample Data:** This section defines example data for the problem:
    *   `items`: A list of item identifiers.
    *   `weights`: A dictionary mapping items to their weights.
    *   `box_capacity`: The maximum weight capacity of each box.
    *   `max_boxes`: The upper limit on the number of boxes available.

3.  **Model Initialization:** `model = pyo.ConcreteModel()` creates a concrete Pyomo model instance.

4.  **Sets:**
    *   `model.I = pyo.Set(initialize=items)` defines a set 'I' representing the items.
    *   `model.J = pyo.RangeSet(1, max_boxes)` defines a set 'J' representing the boxes.

5.  **Parameters:**
    *   `model.w = pyo.Param(model.I, initialize=weights)` defines a parameter 'w' storing the weight of each item.
    *   `model.C = box_capacity` sets the box capacity as a parameter.
    *   `model.N = max_boxes` sets the maximum number of boxes as a parameter.

6.  **Variables:**
    *   `model.x = pyo.Var(model.I, model.J, domain=pyo.Binary)` defines binary variable 'x' indicating if item 'i' is assigned to box 'j'.
    *   `model.y = pyo.Var(model.J, domain=pyo.Binary)` defines binary variable 'y' indicating if box 'j' is used.

7.  **Objective Function:** `model.obj = pyo.Objective(expr=sum(model.y[j] for j in model.J), sense=pyo.minimize)` defines the objective to minimize, which is the total number of boxes used.

8.  **Constraints:**
    *   `model.constraint1` ensures that each item is assigned to exactly one box.
    *   `model.constraint2` enforces the weight capacity constraint for each box.
    *   `model.constraint3` limits the total number of boxes used to the maximum allowed.

9.  **Solver:**
    *   `solver = pyo.SolverFactory('glpk')` creates a solver instance using the 'glpk' solver (you might need to install it).
    *   `results = solver.solve(model)` solves the model using the selected solver.

10. **Results:**
    *   The code prints the solver status and termination condition.
    *   It prints the optimal number of boxes used.
    *   It then iterates through the boxes and prints the items assigned to each used box.

**Remember:** Replace the sample data with your actual data to solve your specific problem instance.


## 4. Input problem data and try running the generated code

In [632]:



import pyomo.environ as pyo

# Sample data (replace with your actual data)
items = ['A', 'B', 'C', 'D', 'E']  
weights = {'A': 4, 'B': 2, 'C': 7, 'D': 3, 'E': 5} 
box_capacity = 10
max_boxes = 3

items =  range(1,25)
weights = {1: 2,2: 2,3: 2,4: 2,5: 3,6: 3,7:4,8: 4,9: 4,10: 4,11: 4,12:4,13: 5,14:5,15: 5,16: 5,17: 5,18: 5,19: 6,20:6,21: 7,22: 7,23: 8, 24:8} # Weight of each item
boxes = range(1,15)
box_capacity = 9
max_boxes = 15

# Model
model = pyo.ConcreteModel()

# Sets
model.I = pyo.Set(initialize=items) # set of items
model.J = pyo.RangeSet(1, max_boxes) # set of boxes

# Parameters
model.w = pyo.Param(model.I, initialize=weights)  # weight of each item
model.C = box_capacity # capacity of each box
model.N = max_boxes # maximum number of boxes

# Variables
model.x = pyo.Var(model.I, model.J, domain=pyo.Binary) # 1 if item i is in box j, 0 otherwise
model.y = pyo.Var(model.J, domain=pyo.Binary) # 1 if box j is used, 0 otherwise

# Objective function: Minimize the number of boxes used
model.obj = pyo.Objective(expr=sum(model.y[j] for j in model.J), sense=pyo.minimize)

# Constraints
# Each item must be assigned to exactly one box
model.constraint1 = pyo.ConstraintList()
for i in model.I:
    model.constraint1.add(sum(model.x[i, j] for j in model.J) == 1)

# Weight capacity constraint for each box
model.constraint2 = pyo.ConstraintList()
for j in model.J:
    model.constraint2.add(sum(model.w[i] * model.x[i, j] for i in model.I) <= model.C * model.y[j])

# Limit the total number of boxes used
model.constraint3 = pyo.Constraint(expr=sum(model.y[j] for j in model.J) <= model.N)

# Solve the model
solver = pyo.SolverFactory('glpk') 
results = solver.solve(model)

# Print the results
print("Status:", results.solver.status)
print("Termination condition:",results.solver.termination_condition)
print("Optimal number of boxes:", pyo.value(model.obj))

# Print box assignments
for j in model.J:
    if pyo.value(model.y[j]) > 0:
        print(f"Box {j}:", [i for i in model.I if pyo.value(model.x[i, j]) > 0.5])

Status: ok
Termination condition: optimal
Optimal number of boxes: 13.0
Box 1: [23]
Box 2: [3, 21]
Box 3: [1, 22]
Box 4: [4, 16]
Box 5: [2, 20]
Box 6: [6, 17]
Box 7: [8, 14]
Box 8: [7, 13]
Box 9: [9, 15]
Box 10: [10, 11]
Box 11: [12, 18]
Box 12: [24]
Box 14: [5, 19]


## 5. Correct the code to verify model viability (optional)

## 6. Print the responses

In [633]:
print(response.text)

Let:

*  $I$ be the set of items.
* $J$ be the set of boxes. 
* $w_i$ be the weight of item $i \in I$.
* $C$ be the capacity of each box.
* $N$ be the upper bound on the number of boxes. 
* $x_{ij}$ be a binary variable equal to 1 if item $i \in I$ is assigned to box $j \in J$, and 0 otherwise. 
* $y_j$ be a binary variable equal to 1 if box $j \in J$ is used, and 0 otherwise. 



In [634]:
print(response2.text)

Minimize $\sum_{j \in J} y_j$ 



In [635]:
print(response3.text)

Subject to:

* $\sum_{j \in J} x_{ij} = 1, \forall i \in I$ 
* $\sum_{i \in I} w_i x_{ij}  \leq C y_j, \forall j \in J$
* $\sum_{j \in J} y_j \leq N$
* $x_{ij} \in \{0,1\}, \forall i \in I, \forall j \in J$
* $y_{j} \in \{0,1\}, \forall j \in J$ 



In [636]:
print(response4.text)

```python
import pyomo.environ as pyo

# Sample data (replace with your actual data)
items = ['A', 'B', 'C', 'D', 'E']  
weights = {'A': 4, 'B': 2, 'C': 7, 'D': 3, 'E': 5} 
box_capacity = 10
max_boxes = 3

# Model
model = pyo.ConcreteModel()

# Sets
model.I = pyo.Set(initialize=items) # set of items
model.J = pyo.RangeSet(1, max_boxes) # set of boxes

# Parameters
model.w = pyo.Param(model.I, initialize=weights)  # weight of each item
model.C = box_capacity # capacity of each box
model.N = max_boxes # maximum number of boxes

# Variables
model.x = pyo.Var(model.I, model.J, domain=pyo.Binary) # 1 if item i is in box j, 0 otherwise
model.y = pyo.Var(model.J, domain=pyo.Binary) # 1 if box j is used, 0 otherwise

# Objective function: Minimize the number of boxes used
model.obj = pyo.Objective(expr=sum(model.y[j] for j in model.J), sense=pyo.minimize)

# Constraints
# Each item must be assigned to exactly one box
model.constraint1 = pyo.ConstraintList()
for i in model.I:
    model.constrai